# Bronze-to-Silver Agent Flow

## Overview
This notebook demonstrates the **data cleaning process** of the Agentic ELT Pipeline.
The Silver Layer transforms raw Bronze data into cleaned, standardized tables.

### Agent Workflow:
1. **Silver Draft Agent** → Analyzes data quality and plans transformations
2. **Silver Builder Agent** → Generates executable Python code
3. **Silver Runner** → Executes cleaning and creates Silver tables

### What you'll learn:
- How LLM agents automatically detect data quality issues
- Which transformations are applied for data cleaning
- How agents generate code for ETL processes

## 📋 Setup and Configuration

**What happens here:**
- Load required Python libraries
- Define paths to Bronze and Silver artifacts
- Verify data availability

In [1]:
import json
import pandas as pd
from pathlib import Path
import yaml

# Set your Bronze and Silver Run IDs here
# Note: Silver creates its own run_id with new timestamp but same suffix
bronze_run_id = "20260124_231016_#6470e523"
silver_run_id = "20260124_231151_#6470e523"  # Update this to match your Silver run

# Define paths
bronze_dir = Path(f"../artifacts/bronze/{bronze_run_id}")
silver_dir = Path(f"../artifacts/silver/{silver_run_id}")

print(f"🔍 Checking data directories:")
print(f"Bronze Directory: {bronze_dir} ({'✅ exists' if bronze_dir.exists() else '❌ missing'})")
print(f"Silver Directory: {silver_dir} ({'✅ exists' if silver_dir.exists() else '❌ missing'})")

🔍 Checking data directories:
Bronze Directory: ..\artifacts\bronze\20260124_231016_#6470e523 (✅ exists)
Silver Directory: ..\artifacts\silver\20260124_231016_#6470e523 (✅ exists)


## 🥉 Bronze Layer Inspection

**What happens here:**
- Analyze raw input data from the Bronze Layer
- Review metadata and data structures
- Identify available tables and their properties

**Why important:**
The Silver Draft Agent needs this information to detect data quality issues and develop transformation strategies.

In [2]:
# Analyze Bronze metadata
bronze_metadata_path = bronze_dir / "data" / "metadata.yaml"
if bronze_metadata_path.exists():
    with open(bronze_metadata_path, 'r') as f:
        bronze_metadata = yaml.safe_load(f)
    
    print("📊 Bronze Layer Metadata:")
    print(f"Run ID: {bronze_metadata.get('run_id')}")
    print(f"Layer: {bronze_metadata.get('layer')}")
    
    summary = bronze_metadata.get('summary', {})
    print(f"\n📈 Ingestion Summary:")
    print(f"  Files total: {summary.get('files_total')}")
    print(f"  Files success: {summary.get('files_success')}")
    print(f"  Files failed: {summary.get('files_failed')}")
else:
    print("❌ Bronze metadata not found")

📊 Bronze Layer Metadata:
Run ID: None
Layer: None

📈 Ingestion Summary:
  Files total: 9
  Files success: 9
  Files failed: 0


In [3]:
# Inspect Bronze tables
bronze_data_dir = bronze_dir / "data"
csv_files = list(bronze_data_dir.glob("*.csv"))

print(f"🗂️ Bronze Tables ({len(csv_files)} found):")
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    print(f"\n📋 {csv_file.name}:")
    print(f"  Dimensions: {df.shape[0]} rows × {df.shape[1]} columns")
    print(f"  Columns: {list(df.columns)}")
    
    # Initial data quality indicators
    null_count = df.isnull().sum().sum()
    duplicate_count = df.duplicated().sum()
    print(f"  Quality: {null_count} NULL values, {duplicate_count} duplicates")

🗂️ Bronze Tables (9 found):

📋 CST_AZ12.csv:
  Dimensions: 18484 rows × 3 columns
  Columns: ['CID', 'BDATE', 'GEN']
  Quality: 1472 NULL values, 0 duplicates

📋 cst_info.csv:
  Dimensions: 18494 rows × 7 columns
  Columns: ['cst_id', 'cst_key', 'cst_firstname', 'cst_lastname', 'cst_marital_status', 'cst_gndr', 'cst_create_date']
  Quality: 4608 NULL values, 0 duplicates

📋 customer_info.csv:
  Dimensions: 5 rows × 5 columns
  Columns: ['customer_id', 'firstname', 'lastname', 'gender', 'date_of_birth']
  Quality: 0 NULL values, 0 duplicates

📋 LOC_A101.csv:
  Dimensions: 18484 rows × 2 columns
  Columns: ['CID', 'CNTRY']
  Quality: 332 NULL values, 0 duplicates

📋 prd_info.csv:
  Dimensions: 397 rows × 7 columns
  Columns: ['prd_id', 'prd_key', 'prd_nm', 'prd_cost', 'prd_line', 'prd_start_dt', 'prd_end_dt']
  Quality: 216 NULL values, 0 duplicates

📋 product_info.csv:
  Dimensions: 4 rows × 4 columns
  Columns: ['product_id', 'product_name', 'category', 'price']
  Quality: 0 NULL value

## 🤖 Silver Draft Agent - Data Profiling

**What happens here:**
- Simulate the automatic data profiling of the Silver Draft Agent
- Detect data quality issues (NULL values, duplicates, formatting errors)
- Generate transformation recommendations

**Agent Logic:**
The Silver Draft Agent analyzes each column for:
- Data type consistency
- Whitespace issues
- Empty strings vs. NULL values
- Row-level duplicates

In [4]:
def simulate_silver_draft_agent(df, filename):
    """Simulates the data profiling of the Silver Draft Agent"""
    profile = {
        "table": filename,
        "row_count": len(df),
        "column_count": len(df.columns),
        "columns": list(df.columns)
    }
    
    # Agent Analysis: Identify transformation needs
    transforms = []
    quality_issues = []
    
    for col in df.columns:
        # Check for whitespace issues
        if df[col].dtype == 'object':
            if df[col].astype(str).str.strip().ne(df[col].astype(str)).any():
                transforms.append(f"🧹 Trim whitespace in '{col}'")
                quality_issues.append(f"Whitespace in {col}")
            
            # Check for empty strings
            if (df[col] == "").any():
                transforms.append(f"🔄 Convert empty strings to NULL in '{col}'")
                quality_issues.append(f"Empty strings in {col}")
        
        # Check for NULL values
        null_count = df[col].isnull().sum()
        if null_count > 0:
            transforms.append(f"⚠️ Handle {null_count} NULL values in '{col}'")
            quality_issues.append(f"{null_count} NULLs in {col}")
    
    # Check for duplicates
    duplicate_count = df.duplicated().sum()
    if duplicate_count > 0:
        transforms.append(f"🗑️ Remove {duplicate_count} duplicate rows")
        quality_issues.append(f"{duplicate_count} duplicate rows")
    
    profile["quality_issues"] = quality_issues
    profile["suggested_transforms"] = transforms if transforms else ["✅ No transformations needed"]
    
    return profile

# Run agent simulation for all tables
print("🤖 Silver Draft Agent Analysis:")
table_profiles = {}

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    profile = simulate_silver_draft_agent(df, csv_file.name)
    table_profiles[csv_file.name] = profile
    
    print(f"\n📊 Analysis: {csv_file.name}")
    print(f"  Quality issues: {len(profile['quality_issues'])}")
    for issue in profile['quality_issues']:
        print(f"    - {issue}")
    
    print(f"  Transformation plan:")
    for transform in profile['suggested_transforms']:
        print(f"    - {transform}")

🤖 Silver Draft Agent Analysis:

📊 Analysis: CST_AZ12.csv
  Quality issues: 2
    - Whitespace in GEN
    - 1472 NULLs in GEN
  Transformation plan:
    - 🧹 Trim whitespace in 'GEN'
    - ⚠️ Handle 1472 NULL values in 'GEN'

📊 Analysis: cst_info.csv
  Quality issues: 8
    - 4 NULLs in cst_id
    - Whitespace in cst_firstname
    - 8 NULLs in cst_firstname
    - Whitespace in cst_lastname
    - 7 NULLs in cst_lastname
    - 7 NULLs in cst_marital_status
    - 4578 NULLs in cst_gndr
    - 4 NULLs in cst_create_date
  Transformation plan:
    - ⚠️ Handle 4 NULL values in 'cst_id'
    - 🧹 Trim whitespace in 'cst_firstname'
    - ⚠️ Handle 8 NULL values in 'cst_firstname'
    - 🧹 Trim whitespace in 'cst_lastname'
    - ⚠️ Handle 7 NULL values in 'cst_lastname'
    - ⚠️ Handle 7 NULL values in 'cst_marital_status'
    - ⚠️ Handle 4578 NULL values in 'cst_gndr'
    - ⚠️ Handle 4 NULL values in 'cst_create_date'

📊 Analysis: customer_info.csv
  Quality issues: 0
  Transformation plan:
    - ✅ 

## 📄 Silver Agent Context - LLM Output

**What happens here:**
- Inspect the actual output of the Silver Draft Agent
- Review LLM-generated analysis and recommendations
- Compare simulated vs. real agent analysis

**Agent Output:**
The Silver Draft Agent creates structured JSON context data that the Silver Builder Agent uses to generate executable code.

In [ ]:
# Analyze Silver Agent Context
silver_context_path = silver_dir / "reports" / "silver_run_agent_context.json"
if silver_context_path.exists():
    with open(silver_context_path, 'r') as f:
        silver_context = json.load(f)
    
    print("🤖 Silver Draft Agent - LLM Output:")
    print(f"Run ID: {silver_context.get('run_id')}")
    print(f"Layer: {silver_context.get('layer')}")
    print(f"Source Layer: {silver_context.get('source_layer')}")
    
    # Performance metrics
    perf = silver_context.get('performance_metrics', {})
    print(f"\n📈 Agent Performance:")
    print(f"  Rows processed: {perf.get('total_rows_processed', 0):,}")
    print(f"  Columns processed: {perf.get('total_columns_processed', 0):,}")
    print(f"  Tables processed: {perf.get('tables_processed', 0)}")
    print(f"  Tables failed: {perf.get('tables_failed', 0)}")
    
    # Schema overview from agent
    if 'profile' in silver_context:
        print(f"\n🔍 Agent Schema Analysis:")
        schema = silver_context['profile'].get('schema_overview', {})
        for table in schema.get('tables', []):
            print(f"  📋 {table['table']}: {table['row_count']} rows, {table['column_count']} columns")
else:
    print("❌ Silver Agent Context not found - Agent was not executed")

## 📝 Silver Human Report - LLM Insights

**What happens here:**
- Display the human-readable report from the Silver Draft Agent
- Review LLM-generated data quality analysis
- Understand agent recommendations for business stakeholders

**Report Content:**
The agent creates a structured Markdown report with executive summary, data quality assessment, and transformation recommendations.

In [ ]:
# Inspect Silver Human Report
silver_report_path = silver_dir / "reports" / "silver_run_human_report.md"
if silver_report_path.exists():
    with open(silver_report_path, 'r', encoding='utf-8') as f:
        silver_report = f.read()
    
    print("📄 Silver Draft Agent - Human Report:")
    print("=" * 60)
    
    # Show first part of report
    lines = silver_report.split('\n')
    for i, line in enumerate(lines[:30]):  # First 30 lines
        print(line)
    
    if len(lines) > 30:
        print(f"\n... ({len(lines)-30} more lines)")
    
    print(f"\n📊 Report Statistics:")
    print(f"  Total length: {len(silver_report):,} characters")
    print(f"  Lines: {len(lines)}")
    print(f"  Sections: {silver_report.count('#')} (estimated)")
else:
    print("❌ Silver Human Report not found")

## 🥈 Silver Layer Results

**What happens here:**
- Inspect the transformed Silver data
- Compare Bronze vs. Silver versions of tables
- Verify data quality improvements

**Transformation Results:**
The Silver Runner has executed the transformations generated by the Silver Builder Agent and created cleaned data.

In [ ]:
# Analyze Silver transformation results
silver_data_dir = silver_dir / "data"
if silver_data_dir.exists():
    silver_csv_files = list(silver_data_dir.glob("*.csv"))
    
    print(f"🥈 Silver Layer Results ({len(silver_csv_files)} tables):")
    
    for csv_file in silver_csv_files:
        df_silver = pd.read_csv(csv_file)
        print(f"\n📊 {csv_file.name}:")
        print(f"  Silver: {df_silver.shape[0]} rows × {df_silver.shape[1]} columns")
        
        # Compare with Bronze version
        bronze_equivalent = bronze_data_dir / csv_file.name
        if bronze_equivalent.exists():
            df_bronze = pd.read_csv(bronze_equivalent)
            
            # Data quality improvements
            bronze_nulls = df_bronze.isnull().sum().sum()
            silver_nulls = df_silver.isnull().sum().sum()
            bronze_dupes = df_bronze.duplicated().sum()
            silver_dupes = df_silver.duplicated().sum()
            
            print(f"  📈 Quality improvements:")
            print(f"    Rows: {df_bronze.shape[0]} → {df_silver.shape[0]} ({df_silver.shape[0] - df_bronze.shape[0]:+d})")
            print(f"    NULL values: {bronze_nulls} → {silver_nulls} ({silver_nulls - bronze_nulls:+d})")
            print(f"    Duplicates: {bronze_dupes} → {silver_dupes} ({silver_dupes - bronze_dupes:+d})")
            
            # Data type improvements
            print(f"  🔄 Data type transformations:")
            for col in df_silver.columns:
                if col in df_bronze.columns:
                    bronze_type = str(df_bronze[col].dtype)
                    silver_type = str(df_silver[col].dtype)
                    if bronze_type != silver_type:
                        print(f"    {col}: {bronze_type} → {silver_type}")
        else:
            print(f"  ⚠️ No Bronze comparison data found")
else:
    print("❌ Silver data not found - Transformation not executed")

## 📊 Data Quality Dashboard

**What happens here:**
- Create overview of data quality improvements
- Quantify agent performance
- Assess transformation success

**Quality Metrics:**
Compare data quality before and after Silver transformation.

In [ ]:
# Create data quality dashboard
if silver_data_dir.exists() and bronze_data_dir.exists():
    print("📊 DATA QUALITY DASHBOARD")
    print("=" * 50)
    
    total_bronze_rows = 0
    total_silver_rows = 0
    total_bronze_nulls = 0
    total_silver_nulls = 0
    total_bronze_dupes = 0
    total_silver_dupes = 0
    tables_processed = 0
    
    for csv_file in silver_csv_files:
        bronze_file = bronze_data_dir / csv_file.name
        if bronze_file.exists():
            df_bronze = pd.read_csv(bronze_file)
            df_silver = pd.read_csv(csv_file)
            
            total_bronze_rows += len(df_bronze)
            total_silver_rows += len(df_silver)
            total_bronze_nulls += df_bronze.isnull().sum().sum()
            total_silver_nulls += df_silver.isnull().sum().sum()
            total_bronze_dupes += df_bronze.duplicated().sum()
            total_silver_dupes += df_silver.duplicated().sum()
            tables_processed += 1
    
    print(f"📈 OVERALL STATISTICS:")
    print(f"  Tables processed: {tables_processed}")
    print(f"  Rows: {total_bronze_rows:,} → {total_silver_rows:,} ({total_silver_rows - total_bronze_rows:+,})")
    print(f"  NULL values: {total_bronze_nulls:,} → {total_silver_nulls:,} ({total_silver_nulls - total_bronze_nulls:+,})")
    print(f"  Duplicates: {total_bronze_dupes:,} → {total_silver_dupes:,} ({total_silver_dupes - total_bronze_dupes:+,})")
    
    # Calculate quality score
    if total_bronze_rows > 0:
        bronze_quality = (1 - (total_bronze_nulls + total_bronze_dupes) / (total_bronze_rows * 10)) * 100
        silver_quality = (1 - (total_silver_nulls + total_silver_dupes) / (total_silver_rows * 10)) * 100
        quality_improvement = silver_quality - bronze_quality
        
        print(f"\n🎯 QUALITY SCORE:")
        print(f"  Bronze Layer: {bronze_quality:.1f}%")
        print(f"  Silver Layer: {silver_quality:.1f}%")
        print(f"  Improvement: {quality_improvement:+.1f}%")
else:
    print("❌ Cannot create data quality dashboard - Data missing")

## 🔄 Agent Workflow Summary

**What you learned:**
This section summarizes the entire Silver Agent workflow and explains the role of each agent in the data cleaning process.

In [ ]:
print("🤖 SILVER AGENT WORKFLOW - SUMMARY")
print("=" * 60)
print()
print("1️⃣ SILVER DRAFT AGENT (Analysis & Planning):")
print("   🔍 Reads Bronze metadata and CSV files")
print("   📊 Performs automatic data profiling")
print("   ⚠️ Identifies data quality issues")
print("   💡 Suggests specific transformations")
print("   📄 Creates silver_run_agent_context.json")
print("   📝 Creates silver_run_human_report.md")
print()
print("2️⃣ SILVER BUILDER AGENT (Code Generation):")
print("   📖 Reads Silver Draft Context")
print("   🛠️ Generates executable Python code")
print("   🔧 Implements suggested transformations")
print("   📜 Creates load_2_silver_layer_runner.py")
print()
print("3️⃣ SILVER RUNNER (Execution):")
print("   ▶️ Executes generated code")
print("   🔄 Transforms Bronze → Silver")
print("   💾 Creates cleaned CSV files")
print("   📋 Generates metadata and reports")
print()
print("✅ RESULT: Cleaned, standardized Silver-Layer data")
print("   - Consistent data types")
print("   - Removed duplicates")
print("   - Standardized NULL values")
print("   - Clean formatting")
print("   - Complete documentation")